In [ ]:
!pip install openai
!pip install pinecone-client
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 24.3 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import openai

openai_api_key = userdata.get('openai-secret')
pinecone_api_key = userdata.get('pinecone-api-key')

#1. Extract PDF file and convert text into Embeddings

In [ ]:
import fitz
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    texts = [page.get_text() for page in doc]
    doc.close()
    return texts

In [ ]:
from openai import OpenAI

def get_embeddings(text):
    client = OpenAI(api_key=openai_api_key)
    response = client.embeddings.create(input=text,model="text-embedding-ada-002")
    return response.data[0].embedding

In [ ]:
import pandas as pd
import uuid
import json

def create_dataframe(texts):
    data = {
        "id": [],
        "embeddings_vector": [],
        "metadata": []
    }

    for i, text in enumerate(texts):
        data["id"].append(str(uuid.uuid4()))
        data["embeddings_vector"].append(get_embeddings(text))
        data["metadata"].append(json.dumps({"chunk": i, "page": i + 1, "text": text}))

    return pd.DataFrame(data)

In [ ]:
texts = extract_text_from_pdf('/content/LISR-2024.Articulo5.pdf')

df = create_dataframe(texts)
df.head()

,id,embeddings_vector,metadata
0,fbc9038e-9811-4857-8840-95b05706e37f,"[-0.01628805883228779, -0.010170447640120983, ...","{""chunk"": 0, ""page"": 1, ""text"": ""Art\u00edculo..."
1,8a294d38-aa76-4433-b670-96cd1c1c5af6,"[-0.02239667810499668, -0.014409237541258335, ...","{""chunk"": 1, ""page"": 2, ""text"": ""U: Utilidad q..."
2,83b9bf3b-46b5-4185-b004-636a2a5736fd,"[-0.02847621776163578, -0.025382699444890022, ...","{""chunk"": 2, ""page"": 3, ""text"": ""M\u00e9xico, ..."
3,249f5580-3cec-4912-b701-247118eefa9b,"[-0.016880128532648087, -0.010336634702980518,...","{""chunk"": 3, ""page"": 4, ""text"": ""l\u00edmite d..."
4,6c2232ca-86b8-4086-b027-1b308a6251f9,"[-0.012105986475944519, 0.0012338667875155807,...","{""chunk"": 4, ""page"": 5, ""text"": ""exceder\u00e1..."


#2. Upsert the text embeddings into Pinecone index

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("tech16llm01")
index

#2.1 Testing vector queries to pinecone

In [ ]:
import json

vectors = []
for i, row in df.iterrows():
  vectors.append({
      "id": row['id'],
      "values": row['embeddings_vector'],
      "metadata": json.loads(row['metadata'])
  });

index.upsert(vectors, namespace='lisr-2024')

{'upserted_count': 6}

In [ ]:
query = "Where the lions live in Africa?"

query_embeddings = get_embeddings(query)
query_result = index.query(vector=query_embeddings, top_k=3, include_metadata=True, namespace = 'lisr-2024')
relevant_results = [result for result in query_result['matches'] if result.score >= 0.85]

print(relevant_results)


[]


In [ ]:
query = "¿Qué impuesto pueden acreditar los residentes en México contra el impuesto que les corresponda pagar?"

query_embeddings = get_embeddings(query)
query_result = index.query(vector=query_embeddings, top_k=3, include_metadata=True, namespace = 'lisr-2024')
relevant_results = [result for result in query_result['matches'] if result.score >= 0.85]

print(relevant_results)

[{'id': '7f6a822a-03b7-462a-884b-e3e4420ef2fa',
 'metadata': {'chunk': 0.0,
              'page': 1.0,
              'text': 'Artículo 5. Los residentes en México podrán acreditar, '
                      'contra el impuesto que conforme a \n'
                      'esta Ley les corresponda pagar, el impuesto sobre la '
                      'renta que hayan pagado en el \n'
                      'extranjero por los ingresos procedentes de fuente '
                      'ubicada en el extranjero, siempre que se \n'
                      'trate de ingresos por los que se esté obligado al pago '
                      'del impuesto en los términos de la \n'
                      'presente Ley. El acreditamiento a que se refiere este '
                      'párrafo sólo procederá siempre que el \n'
                      'ingreso acumulado, percibido o devengado, incluya el '
                      'impuesto sobre la renta pagado en \n'
                      'el extranjero.  \n'
           

#3. Defining openAI chat completion TOOLS

In [ ]:
isr2024_article5_tools = [
    {
        "type":"function",
        "function": {
            "name": "compute_mpi",
            "description": """Use this function to compute: el monto proporcional del impuesto sobre la renta pagado en el extranjero por la sociedad residente en otro país correspondiente
            al dividendo o utilidad percibido por la persona moral residente en México""",
            "parameters": {
                "type": "object",
                "properties": {
                    "d": {
                        "type": "number",
                        "description": f"""
                                Dividendo o utilidad distribuido por la sociedad residente en el extranjero a la persona moral residente en México sin disminuir la retención o pago del
                                impuesto sobre la renta que en su caso se haya efectuado por su distribución.
                                """,
                    },
                    "u": {
                        "type": "number",
                        "description": f"""
                                Utilidad que sirvió de base para repartir los dividendos, después del pago del impuesto sobre la renta en primer nivel corporativo, obtenida por la sociedad
                                residente en el extranjero que distribuye dividendos a la persona moral residente en México.
                                """,
                    },
                    "ic": {
                        "type": "number",
                        "description": f"""
                                Impuesto sobre la renta corporativo pagado en el extranjero por la sociedad residente en el extranjero que distribuyó dividendos a la persona moral residente
                                en México.
                                """,
                    }
                },
                "required": ["d","u","ic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "compute_mpi_second_level",
            "description": """Use this function to compute: Monto proporcional del impuesto sobre la renta pagado en el extranjero por la sociedad residente en el extranjero en segundo nivel
            corporativo, que distribuye dividendos o utilidades a la otra sociedad extranjera en primer nivel corporativo, que a su vez distribuye dividendos o utilidades a la persona moral
            residente en México""",
            "parameters": {
                "type": "object",
                "properties": {
                    "d": {
                        "type": "number",
                        "description": f"""
                                Dividendo o utilidad distribuido por la sociedad residente en el extranjero a la persona moral residente en México sin disminuir la retención o pago del
                                impuesto sobre la renta que en su caso se haya efectuado por su distribución.
                                """,
                    },
                    "u": {
                        "type": "number",
                        "description": f"""
                                Utilidad que sirvió de base para repartir los dividendos, después del pago del impuesto sobre la renta en primer nivel corporativo,
                                obtenida por la sociedad residente en el extranjero que distribuye dividendos a la persona moral residente en México.
                                """,
                    },
                    "d2": {
                        "type": "number",
                        "description": f"""
                                Dividendo o utilidad distribuida por la sociedad residente en el extranjero a la sociedad residente en el extranjero que distribuye dividendos a la persona moral
                                residente en México, sin disminuir la retención o pago del impuesto sobre la renta que en su caso se haya efectuado por la primera distribución.
                                """,
                    },
                    "u2": {
                        "type": "number",
                        "description": f"""
                                Utilidad que sirvió de base para repartir los dividendos después del pago del impuesto sobre la renta en segundo nivel corporativo, obtenida por la sociedad residente
                                en el extranjero que distribuye dividendos a la otra sociedad residente en el extranjero que distribuye dividendos a la persona moral residente en México.
                                """,
                    },
                    "ic2": {
                        "type": "number",
                        "description": f"""
                                Impuesto sobre la renta corporativo pagado en el extranjero por la sociedad residente en el extranjero que distribuyó dividendos a la otra sociedad residente en el
                                extranjero que distribuye dividendos a la persona moral residente en México
                                """,
                    }
                },
                "required": ["d","u","d2","u2","ic2"]
            }
        }
    },
    {
        "type":"function",
        "function": {
            "name": "compute_la",
            "description": """Use this function to compute: Límite de acreditamiento por los impuestos sobre la renta corporativos pagados en el extranjero en primer y segundo nivel corporativo.
            """,
            "parameters": {
                "type": "object",
                "properties": {
                    "d": {
                        "type": "number",
                        "description": f"""
                                Dividendo o utilidad distribuido por la sociedad residente en el extranjero a la persona moral residente en México sin disminuir la retención o pago del impuesto
                                sobre la renta que en su caso se haya efectuado por su distribución.
                                """,
                    },
                    "mpi": {
                        "type": "number",
                        "description": f"""
                                Monto proporcional del impuesto sobre la renta corporativo pagado en el extranjero a que se refiere el tercer párrafo de este artículo
                                """,
                    },
                    "mpi2": {
                        "type": "number",
                        "description": f"""
                                Monto proporcional del impuesto sobre la renta corporativo pagado en el extranjero a que se refiere el cuarto párrafo de este artículo.
                                """,
                    },
                    "t": {
                        "type": "number",
                        "description": f"""
                                Tasa (numero decimal de 0.1 a 1.0)
                                """,
                    },
                    "id": {
                        "type": "number",
                        "description": f"""
                                Impuesto acreditable a que se refiere el primer y sexto párrafos de este artículo que corresponda al dividendo o utilidad percibido por la persona moral residente en México.
                                """,
                    }
                },
                "required": ["d","mpi","mpi2","t","id"]
            }
        }
    }
]

#4. Defining functions to address the RAG with TOOLS chat completion workflow using openAI chatGPT models

In [ ]:
def get_rag_relevant_info(textQuery):
  query_embeddings = get_embeddings(textQuery)
  query_result = index.query(vector=query_embeddings, top_k=3, include_metadata=True, namespace = 'lisr-2024')
  relevant_results = [result for result in query_result['matches'] if result.score >= 0.85]

  return relevant_results

In [ ]:
def get_augmented_query(textQuery):
  relevant_results = get_rag_relevant_info(textQuery)

  print(f"Relevant results for {textQuery}: {relevant_results}")

  augmented_query = textQuery
  if len(relevant_results) > 0:
    contexts = [item['metadata']['text'] for item in relevant_results]
    augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n" + textQuery

    print(f"Augmented query: {augmented_query}")

  return augmented_query, relevant_results

In [ ]:
from openai import OpenAI

def invoke_openai_chat_completion(query):
  primer = f"""You are Q&A bot. A highly intelligent system that answers
            user questions based on the information provided by the user above
            each question. If the information can not be found in the information
            provided by the user you truthfully can try to answer the question
            with the knowledge you have, but letting know to the user that there was not information provided.
            """
  augmented_query, relevant_results = get_augmented_query(query)
  messages=[
    {"role": "system", "content": primer},
    {"role": "user", "content": augmented_query}
  ]

  client = OpenAI(api_key=openai_api_key)
  response = None

  if len(relevant_results) > 0:
    response = client.chat.completions.create(model="gpt-4", messages = messages, tools = isr2024_article5_tools)
  else:
    response = client.chat.completions.create(model="gpt-4", messages = messages)

  return response


#5.1 Test case 1. A question not related to the RAG context

In [ ]:
from IPython.display import Markdown

query1 = 'Where the lions live in Africa?'

response1 = invoke_openai_chat_completion(query1)
display(Markdown(response1.choices[0].message.content))

Relevant results for Where the lions live in Africa?: []


Lions are found throughout the southern part of the African continent, particularly in the savanna and grassland regions, including countries such as South Africa, Botswana, Tanzania, and Kenya. However, the specific location of lions within Africa would depend on the exact context or information provided. If more specific details were given, I could provide a more precise answer.

#5.2 Test case 2. Question related to the RAG context, but not related to the tools

In [ ]:
query2 = "¿Qué impuesto pueden acreditar los residentes en México contra el impuesto que les corresponda pagar?"

response2 = invoke_openai_chat_completion(query2)
print('-' * 80)
print(response2)
print('-' * 80)
display(Markdown(response2.choices[0].message.content))

Relevant results for ¿Qué impuesto pueden acreditar los residentes en México contra el impuesto que les corresponda pagar?: [{'id': '7f6a822a-03b7-462a-884b-e3e4420ef2fa',
 'metadata': {'chunk': 0.0,
              'page': 1.0,
              'text': 'Artículo 5. Los residentes en México podrán acreditar, '
                      'contra el impuesto que conforme a \n'
                      'esta Ley les corresponda pagar, el impuesto sobre la '
                      'renta que hayan pagado en el \n'
                      'extranjero por los ingresos procedentes de fuente '
                      'ubicada en el extranjero, siempre que se \n'
                      'trate de ingresos por los que se esté obligado al pago '
                      'del impuesto en los términos de la \n'
                      'presente Ley. El acreditamiento a que se refiere este '
                      'párrafo sólo procederá siempre que el \n'
                      'ingreso acumulado, percibido o devengado, incl

Los residentes en México pueden acreditar contra el impuesto que les corresponda pagar, el impuesto sobre la renta que hayan pagado en el extranjero por los ingresos procedentes de fuente ubicada en el extranjero. Este acreditamiento solo procede siempre que el ingreso acumulado, percibido o devengado, incluya el impuesto sobre la renta pagado en el extranjero y se trate de ingresos por los que se esté obligado al pago del impuesto en los términos de la presente Ley.

#5.3 Test case 3. Question related to the RAG context, and related to the TOOLS.

In [ ]:
query3 = "Podrías ayudarme a entender cómo calcular el monto proporcional del impuesto sobre la renta pagado en el extranjero por la sociedad residente en el extranjero en primer nivel corporativo"

response3 = invoke_openai_chat_completion(query3)
print('-' * 80)
print(response3)
print('-' * 80)
display(Markdown(response3.choices[0].message.content))

Relevant results for Podrías ayudarme a entender cómo calcular el monto proporcional del impuesto sobre la renta pagado en el extranjero por la sociedad residente en el extranjero en primer nivel corporativo: [{'id': '23c1eabc-ca7d-41a7-adf5-6a80bfd2657a',
 'metadata': {'chunk': 2.0,
              'page': 3.0,
              'text': 'México, sin disminuir la retención o pago del impuesto '
                      'sobre la renta que en su caso se \n'
                      'haya efectuado por la primera distribución.  \n'
                      'U2: Utilidad que sirvió de base para repartir los '
                      'dividendos después del pago del impuesto \n'
                      'sobre la renta en segundo nivel corporativo, obtenida '
                      'por la sociedad residente en el \n'
                      'extranjero que distribuye dividendos a la otra sociedad '
                      'residente en el extranjero que \n'
                      'distribuye dividendos a la person

<IPython.core.display.Markdown object>

#5.4 Test case 4. Query related to compute something, but not related to the RAG context, neither to the TOOLS.

In [ ]:
query4 = "Podrías ayudarme a calcular los gradientes en una función de optimización Adam"

response4 = invoke_openai_chat_completion(query4)
print('-' * 80)
print(response4)
print('-' * 80)
display(Markdown(response4.choices[0].message.content))

Relevant results for Podrías ayudarme a calcular los gradientes en una función de optimización Adam: []
--------------------------------------------------------------------------------
ChatCompletion(id='chatcmpl-93qhLfAxd0cOQvfJgjN2RUKqyKIW5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Lo siento, no has proporcionado información específica sobre la función o datos con los que estás trabajando. Sin embargo, puedo proporcionarte una descripción general de cómo el optimizador Adam calcula los gradientes.\n\nAdam, que significa "Adaptive Moment Estimation", es un método de optimización de gradiente descendente estocástico que se basa en ajustar adaptativamente la tasa de aprendizaje de cada parámetro en función de la estimación de momentos del primer y segundo orden.\n\nAdam realiza actualizaciones basadas en el promedio móvil exponencial de los gradientes:\n1. Calcula el gradiente de la función de pérdida con respecto a cada parám

Lo siento, no has proporcionado información específica sobre la función o datos con los que estás trabajando. Sin embargo, puedo proporcionarte una descripción general de cómo el optimizador Adam calcula los gradientes.

Adam, que significa "Adaptive Moment Estimation", es un método de optimización de gradiente descendente estocástico que se basa en ajustar adaptativamente la tasa de aprendizaje de cada parámetro en función de la estimación de momentos del primer y segundo orden.

Adam realiza actualizaciones basadas en el promedio móvil exponencial de los gradientes:
1. Calcula el gradiente de la función de pérdida con respecto a cada parámetro en la etapa actual.
2. Calcula el promedio de gradientes recientes.
3. Calcula el promedio de los cuadrados de gradientes recientes.
4. Ajusta los gradientes en función de estos promedios.

Para calcular los gradientes, Adam utiliza dos variables:
- "m" es la media móvil (primer momento) de los gradientes.
- "v" es la media móvil (segundo momento) de los cuadrados de los gradientes.

El procedimiento básico para una única iteración de Adam puede ser como sigue:
1. Calcule el gradiente de la pérdida con respecto a los parámetros (dW, db).
2. Actualice las estimaciones de los momentos (m, v).
3. Corrija las estimaciones de los momentos (m_hat, v_hat).
4. Actualice los parámetros.

Tenga en cuenta que todas las operaciones son elemento a elemento en los pasos anteriores.

Recuerde que no puedo brindarte una respuesta más específica o un código sin detalles adicionales. También vale la pena mencionar que, aunque he proporcionado los fundamentos, los algoritmos de optimización como Adam tienen sutilezas que pueden variar de una implementación a otra.